In [25]:
import pandas as pd
import numpy as np

estb_path = '/Users/koacow/BOSTON UNIVERSITY Dropbox/Ngoc Duy Khoa Cao/Climate Risk and Labor Market/RAIS data/firms and cities/ESTB/treated/estb_2003_treated.csv'
cols = ['cnpj_raiz', 'cnpj_cei', 'year', 'municipio', 'vinculos_ativos']
df = pd.read_csv(estb_path, usecols=cols, encoding='latin1')
df

/var/folders/zx/x6hb1_p95p33hh3235qrnlqm0000gn/T/ipykernel_87932/3736470946.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(estb_path, usecols=cols, encoding='latin1')


,cnpj_cei,cnpj_raiz,municipio,vinculos_ativos,year
0,191,0.0,520005.0,0.0,2003
1,272,0.0,130260.0,76.0,2003
2,353,0.0,150140.0,118.0,2003
3,434,0.0,354850.0,68.0,2003
4,515,0.0,330100.0,100.0,2003
...,...,...,...,...,...
6112774,934046290907,0.0,520330.0,1.0,2003
6112775,941370012181,0.0,291360.0,1.0,2003
6112776,960501958204,0.0,353410.0,1.0,2003
6112777,962911410005,0.0,354980.0,1.0,2003


In [16]:
df[df["vinculos_ativos"] == 0].shape[0] / df.shape[0] * 100

57.73447985900842

In [74]:
# 2. Collapse data to one row per firm-year-municipality level
# emp_city = number of active workers in a firm in a municipality in a year
city_tot = (
    df.groupby(['cnpj_raiz', 'year', 'municipio'], as_index=False)
    .agg(emp_city=('vinculos_ativos', 'sum'))
)
city_tot

,cnpj_raiz,year,municipio,emp_city
0,0.0,2002,110001.0,75.0
1,0.0,2002,110002.0,475.0
2,0.0,2002,110003.0,35.0
3,0.0,2002,110004.0,423.0
4,0.0,2002,110005.0,80.0
...,...,...,...,...
5251489,98941178.0,2002,431810.0,0.0
5251490,99904064.0,2002,330455.0,1.0
5251491,99956880.0,2002,530010.0,0.0
5251492,99990905.0,2002,530010.0,0.0


In [75]:
# 3. Calculate inverse HHI
def calc_hhi(g):
    tot_emp = g['emp_city'].sum()
    if tot_emp == 0:
        return np.nan
    else:
        return (g['emp_city'] ** 2).sum() / (g['emp_city'].sum() ** 2)
hhi = (
  city_tot
    .groupby(['cnpj_raiz', 'year'])
    .apply(calc_hhi)
    .reset_index(name='HHI')
)
hhi['inv_HHI'] = 1 / hhi['HHI']
hhi


/var/folders/zx/x6hb1_p95p33hh3235qrnlqm0000gn/T/ipykernel_62447/1471921793.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calc_hhi)


,cnpj_raiz,year,HHI,inv_HHI
0,0.0,2002,0.002216,451.217764
1,1.0,2002,NaN,NaN
2,2.0,2002,1.000000,1.000000
3,3.0,2002,NaN,NaN
4,6.0,2002,1.000000,1.000000
...,...,...,...,...
5049395,98941178.0,2002,NaN,NaN
5049396,99904064.0,2002,1.000000,1.000000
5049397,99956880.0,2002,NaN,NaN
5049398,99990905.0,2002,NaN,NaN


In [76]:
# 4. Calculate n_municipalities
muni_cnt = (
    city_tot.groupby(['cnpj_raiz', 'year'])
            .municipio.nunique()
            .reset_index(name='n_municipalities')
)
muni_cnt

,cnpj_raiz,year,n_municipalities
0,0.0,2002,4897
1,1.0,2002,1
2,2.0,2002,1
3,3.0,2002,1
4,6.0,2002,1
...,...,...,...
5049395,98941178.0,2002,1
5049396,99904064.0,2002,1
5049397,99956880.0,2002,1
5049398,99990905.0,2002,1


In [26]:
coordinates = pd.read_csv('./geocoded_data/geocoded_data_updt.csv', usecols=['cnpj_cei', 'year', 'lat', 'lng'])
df = df.merge(coordinates, on=['cnpj_cei'], how='left', suffixes=(None, '_y'))
df

,cnpj_cei,cnpj_raiz,municipio,vinculos_ativos,year,year_y,lat,lng
0,191,0.0,520005.0,0.0,2003,NaN,NaN,NaN
1,272,0.0,130260.0,76.0,2003,NaN,NaN,NaN
2,353,0.0,150140.0,118.0,2003,NaN,NaN,NaN
3,434,0.0,354850.0,68.0,2003,NaN,NaN,NaN
4,515,0.0,330100.0,100.0,2003,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
6422782,934046290907,0.0,520330.0,1.0,2003,NaN,NaN,NaN
6422783,941370012181,0.0,291360.0,1.0,2003,NaN,NaN,NaN
6422784,960501958204,0.0,353410.0,1.0,2003,NaN,NaN,NaN
6422785,962911410005,0.0,354980.0,1.0,2003,NaN,NaN,NaN


In [29]:
(1 - df["lat"].isna().sum() / df.shape[0]) * 100

5.687017178056819

In [30]:
df["cnpj_cei"].nunique()

6088502

In [80]:
# 5. Calculate avg_pairwise_distance
from itertools import combinations
from haversine import haversine
import numpy as np
def mean_pair_dist(sub):
    coords = sub[['lat','lng']].drop_duplicates().values
    if len(coords) < 2:
        return 0.0
    dists = [haversine(tuple(a), tuple(b)) for a, b in combinations(coords, 2)]
    return float(np.mean(dists))
dist_df = (
    df.groupby(['cnpj_raiz', 'year'])
      .apply(mean_pair_dist)
      .reset_index(name='avg_pairwise_km')
)
dist_df

KeyboardInterrupt: 

In [ ]:
# 6. Merge all metrics into a single DataFrame
firm_geo_diversity = (
    hhi.merge(muni_cnt, on=['cnpj_raiz', 'year'])
       .merge(dist_df, on=['cnpj_raiz', 'year'])
)
firm_geo_diversity

In [ ]:
firm_geo_diversity.to_csv('./data/firm_geo_diversity.csv', index=False, single_file=True)

In [ ]:
# 7. Sanity checks
assert firm_geo_diversity['HHI'].notnull().all(), "HHI contains null values"
assert firm_geo_diversity['inv_HHI'].notnull().all(), "Inverse HHI contains null values"
assert firm_geo_diversity['n_municipalities'].notnull().all(), "Number of municipalities contains null values"
